## Przykładowe funkcje i parametry i wykorzystywane paczki

In [ ]:
f(x) = sin.(x) .* sqrt.(x)
ReLu(x) = max.(zero.(x),x)
σ(x) = one.(x)./(one.(x).+exp.(-x))
softmax(x) = exp.(x) ./ sum(exp.(x))

x1 = rand(Float64, 10);
x2 = rand(Float64, 100);

using BenchmarkTools
using ForwardDiff

# Różniczkowanie w przód

In [ ]:
struct Dual{T <:Number} <:Number
    v::T
    dv::T
end

In [ ]:
import Base: +, -, *, /
     -(x::Dual) = Dual(-x.v, -x.dv)
     +(x::Dual, y::Dual) = Dual( x.v + y.v, x.dv + y.dv)
     -(x::Dual, y::Dual) = Dual( x.v - y.v, x.dv - y.dv)
     *(x::Dual, y::Dual) = Dual( x.v * y.v, x.dv * y.v + x.v * y.dv)
     /(x::Dual, y::Dual) = Dual( x.v / y.v, (x.dv * y.v - x.v * y.dv)/y.v^2)
 
import Base: abs, sin, cos, tan, exp, sqrt, isless
    abs(x::Dual) = Dual(abs(x.v),sign(x.v)*x.dv)
    sin(x::Dual) = Dual(sin(x.v), cos(x.v)*x.dv)
    cos(x::Dual) = Dual(cos(x.v),-sin(x.v)*x.dv)
    tan(x::Dual) = Dual(tan(x.v), one(x.v)*x.dv + tan(x.v)^2*x.dv)
    exp(x::Dual) = Dual(exp(x.v), exp(x.v)*x.dv)
    sqrt(x::Dual) = Dual(sqrt(x.v),.5/sqrt(x.v) * x.dv)
    isless(x::Dual, y::Dual) = x.v < y.v;

In [ ]:
import Base: show
show(io::IO, x::Dual) = print(io, "(", x.v, ") + [", x.dv, "ϵ]");
value(x::Dual) = x.v;
partials(x::Dual) = x.dv;

In [ ]:
import Base: convert, promote_rule
convert(::Type{Dual{T}}, x::Dual) where T =
 Dual(convert(T, x.v), convert(T, x.dv))
convert(::Type{Dual{T}}, x::Number) where T =
 Dual(convert(T, x), zero(T))
promote_rule(::Type{Dual{T}}, ::Type{R}) where {T,R} =
 Dual{promote_type(T,R)}

## Funkcja obliczająca macierz Jacobiego 

In [ ]:
J = function jacobian(f, args::Vector{T}) where {T <:Number} # przyjmuje jako argumenty funkcje oraz wektor argumentów
    jacobian_columns = Matrix{T}[]
    for i=1:length(args)
        x = Dual{T}[] # tworzy nowy wektor x dla liczb dualnych
        for j=1:length(args)
            seed = (i == j)
            # dodaje do wektora x liczbe dualną z zarodkiem lub bez w zależności od wartości seed 
            push!(x, seed ? Dual(args[j], one(args[j])) : Dual(args[j],zero(args[j])))
            
            #println("x to = ",x)
            #println("iteracja i = ",i," j = ",j)
        end
        #println(f(x))
        column = partials.([f(x)...]) # tworzy wektor pochodnych        
        push!(jacobian_columns, column[:,:]) # i dodaje do macierzy wynikowej
        
    end
    hcat(jacobian_columns...) # zamienia w macierz układając zbiór wektorów w kolumny 
end

## Test macierzy Jacobiego dla różniczkowania w przód

In [ ]:
println("Test f dla wektora x1")
@btime begin
J(f, x1)
end

println("Sprawdzenie ForwardDiff")
@btime begin
ForwardDiff.jacobian(f, x1)   
end

In [ ]:
println("Test f dla wektora x2")
@btime begin
J(f, x2)
end

println("Sprawdzenie ForwardDiff")
@btime begin
ForwardDiff.jacobian(f, x2)   
end

In [ ]:
println("Test ReLu dla wektora x1")
@btime begin
J(ReLu, x1)
end

println("Sprawdzenie ForwardDiff")
@btime begin
ForwardDiff.jacobian(ReLu, x1)
end

In [ ]:
println("Test ReLu dla wektora x2")
@btime begin
J(ReLu, x2)
end

println("Sprawdzenie ForwardDiff")
@btime begin
ForwardDiff.jacobian(ReLu, x2)   
end

In [ ]:
println("Test σ dla wektora x1")
@btime begin
J(σ, x1)
end

println("Sprawdzenie ForwardDiff")
@btime begin
ForwardDiff.jacobian(σ, x1)   
end

In [ ]:
println("Test σ dla wektora x2")
@btime begin
J(σ, x2)
end

println("Sprawdzenie ForwardDiff")
@btime begin
ForwardDiff.jacobian(σ, x2)   
end

# Różniczkowanie w tył

In [ ]:
struct Operator{T}
    f::T
end

abstract type Node end
abstract type LeafNode <: Node end

mutable struct Variable{T} <: LeafNode
    value::T
    grad::T
    
    Variable(val::T) where T = new{T}(val, zero(val))
    Variable(val::T, grad::T) where T = new{T}(val, grad)
end

struct ComputableNode{OT, AT} <: Node
    op::OT # operacja jaka będzie wykonywana na node
    args::AT # argumenty operacji
end

mutable struct CachedNode{NT, OUT} <: Node
    node::NT # ComputableNode
    out::OUT # wynik operacji na zadanych argumentach w computable Node
end

In [ ]:
# TODO: mapowanie int/float na Variable

In [ ]:
# rejestracja operacji z wyrażenia wejściowego
function register(op, args...)
    concreteOp = Operator(op)
    node = ComputableNode(concreteOp, args)
    out = forward(node)
    vvv = CachedNode(node, out)
    CachedNode(node, out)
end

import Base: +, -, *, /
    -(x::Node) = register(-, x)
    +(x::Node, y::Node) = register(+, x, y)
    -(x::Node, y::Node) = register(-, x, y)
    *(x::Node, y::Node) = register(*, x, y)
    /(x::Node, y::Node) = register(/, x, y)
import Base: abs, sin, cos, tan, exp, sqrt, zero, one, max, min, length, sum
    abs(x::Node) = register(abs, x)
    sin(x::Node) = register(sin, x)
    cos(x::Node) = register(cos, x)
    tan(x::Node) = register(tan, x)
    exp(x::Node) = register(exp, x)
    sqrt(x::Node) = register(sqrt, x)
    zero(x::Node) = register(zero, x)
    one(x::Node) = register(one, x)
    max(x::Node, y::Node) = register(max, x, y)
    min(x::Node, y::Node) = register(min, x, y)
    length(x::Node) = register(length, x)
    sum(x::Node) = register(sum, x)

In [ ]:
forward(cached::CachedNode) = cached.out = forward(cached.node)
forward(node::ComputableNode) = forward(node.op, map(forward, node.args)...) # dokonujemy mapowania funkcją forward aby dobrac się do value zmiennych variable
forward(op::Operator, args...) = op.f(args...)  # finalne wykonanie operacji na zadanych argumentach i obliczenie wyniku
forward(var::Variable) = var.value # wykorzystywane do mapowania

In [ ]:
function backward(cached::CachedNode, grad)
    grad_inputs = gradient(cached, grad)
    for (each, each_grad) in zip(cached.node.args, grad_inputs)
        backward(each, each_grad)
    end
end

gradient(cached::CachedNode, grad) =
    gradient(cached.node.op, grad, map(value, cached.node.args)...)

gradient(op::Operator, grad, args...) =
    gradient(op.f, grad, args...)

value(cached::CachedNode) = cached.out
value(var::Variable) = var.value

gradient(::typeof(-), grad, x) = (-grad, )
gradient(::typeof(+), grad, x, y) = (grad, grad)
gradient(::typeof(-), grad, x, y) = (grad, -grad)
gradient(::typeof(*), grad, x, y) = (grad * y, grad * x) # (pochodna po x, pochodna po y)
gradient(::typeof(/), grad, x, y) = (grad * 1/y, grad * -(x / (y * y)))
gradient(::typeof(abs), grad, x) = (grad * x/abs(x), ) 
gradient(::typeof(sin), grad, x) = (grad * cos(x), ) # dodajemy ',' aby cały czas to był tuple
gradient(::typeof(cos), grad, x) = (grad * -sin(x), )
gradient(::typeof(tan), grad, x) = (grad * (tan(x)*tan(x)+1), )
gradient(::typeof(exp), grad, x) = (grad * exp(x), )
gradient(::typeof(sqrt), grad, x) = (grad * 1/(2*sqrt(x)), )
gradient(::typeof(min), grad, x, y) = (isless(x, y) ? grad * one(x) : grad * zero(x), 
                                          isless(x, y) ? grad * zero(y) : grad * one(y))
gradient(::typeof(max), grad, x, y) = (isless(x, y) ? grad * zero(x) : grad * one(x), 
                                          isless(x, y) ? grad * one(y) : grad * zero(y))
gradient(::typeof(zero), grad, x) = (grad * zero(x), ) # zero czy one to po prostu funkcje stałe
gradient(::typeof(one), grad, x) = (grad * zero(x), )
gradient(::typeof(length), grad, x) = (grad * length(x), )
gradient(::typeof(sum), grad, x) = (grad * sum(x), )

function backward(var::Variable, grad)
    var.grad += grad
end

In [ ]:
function jacobian(cached::CachedNode)
    jjj = []
    for each in cached.node.args
        #println("------")
        #println(each)
        x = jacobian(each)
        #println(x)
        push!(jjj, x)
        #println(jjj)
    end
    return jjj
end

function jacobian(var::Variable)
    #println("_________")
    #println(var.grad)
    return var.grad
end

In [ ]:
J2 = function jacobian2(f, args::Vector{T}) where {T <:Number} # przyjmuje jako argumenty funkcje oraz wektor argumentów
    x = map(x -> Variable(x), args)
    y = f(x)
    backward.(y, 1.0)
    diagonal = jacobian.(y)
    size = length(diagonal)
    Jacobian = zeros(size, size)
    for i=1:size
        Jacobian[i,i] = getJacobianValue(diagonal[i])
    end
    #println("jacobian:")
    #println(Jacobian)
end

function getJacobianValue(val::Array{Any})
    return getJacobianValue(val[1])
end

function getJacobianValue(val::Any)
    return val
end

## Testy macierzy Jacobiego dla różniczkowania w tył

In [ ]:
println("Test f dla wektora x1")
@btime begin
J2(f, x1)
end

println("Sprawdzenie ReverseDiff")
using ReverseDiff
@btime begin
ReverseDiff.jacobian(f, x1)
end

In [ ]:
println("Test f dla wektora x2")
@btime begin
J2(f, x2)
end

println("Sprawdzenie ReverseDiff")
using ReverseDiff
@btime begin
ReverseDiff.jacobian(f, x2)
end

In [ ]:
println("Test ReLu dla wektora x1")
@btime begin
J2(ReLu, x1)
end

println("Sprawdzenie ReverseDiff")
using ReverseDiff
@btime begin
ReverseDiff.jacobian(ReLu, x1)
end

In [ ]:
println("Test ReLu dla wektora x2")
@btime begin
J2(ReLu, x2)
end

println("Sprawdzenie ReverseDiff")
using ReverseDiff
@btime begin
ReverseDiff.jacobian(ReLu, x2)
end

In [ ]:
println("Test σ dla wektora x1")
@btime begin
J2(σ, x1)
end

println("Sprawdzenie ReverseDiff")
using ReverseDiff
@btime begin
ReverseDiff.jacobian(σ, x1)
end

In [ ]:
println("Test σ dla wektora x2")
@btime begin
J2(σ, x2)
end

println("Sprawdzenie ReverseDiff")
using ReverseDiff
@btime begin
ReverseDiff.jacobian(σ, x2)
end